<a href="https://colab.research.google.com/github/bl4ckf0xk/ModelX_First_Order/blob/main/ModelX_Model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit, StratifiedKFold, train_test_split, RandomizedSearchCV
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, OrdinalEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report, confusion_matrix, recall_score, precision_score
import joblib

In [2]:
!pip install shap
import shap

In [3]:
DATA_PATH = '/content/drive/MyDrive/Dementia Prediction Dataset.csv'
MODEL_OUT = '/content/dementia_nonmedical_model.pkl'
RANDOM_STATE = 42
TARGET = 'DEMENTED'
SUBJECT_ID = 'NACCID'  # change if your identifier column has a different name

In [4]:
def load_data(path):
    if not os.path.exists(path):
        raise FileNotFoundError(f"Data file not found at {path}. Upload it to Colab or set DATA_PATH correctly.")
    df = pd.read_csv(path)
    return df

In [5]:
NON_MEDICAL_WHITELIST = [
    # Subject demographics
    'BIRTHYR', 'BIRTHMO', 'SEX', 'HISPANIC', 'HISPOR', 'HISPORX',
    'RACE', 'RACEX', 'RACESEC', 'RACESECX', 'RACETER', 'RACETERX',
    'EDUC', 'MARISTAT', 'PRIMLANG', 'PRIMLANX', 'RESIDENC', 'HANDED', 'NACCLIVS', 'INDEPEND',
    # Co-participant
    'INBIRYR', 'INBIRMO', 'INSEX', 'INHISP', 'INHISPOR', 'INHISPOX', 'INRACE', 'INRACEX', 'INRASEC', 'INRASECX', 'INRATER', 'INRATERX', 'INEDUC', 'INRELTO', 'INRELTOX', 'NEWINF',
    # Lifestyle
    'TOBAC30', 'TOBAC100', 'SMOKYRS', 'PACKSPER', 'QUITSMOK', 'ALCOCCAS', 'ALCFREQ',
    # Visit metadata
    'NACCVNUM', 'NACCNVST', 'NACCAVST', 'NACCDAYS', 'NACCFDYS', 'PACKET', 'FORMVER', 'TELCOV', 'TELMOD',
    # Family history (non-genetic fields)
    'NACCFAM', 'NACCMOM', 'NACCDAD'
]

In [6]:
print('Loading data...')
df = load_data(DATA_PATH)
print('Rows:', len(df), 'Columns:', len(df.columns))

Loading data...


/tmp/ipython-input-175837067.py:4: DtypeWarning: Columns (20,22,24,26,28,41,44,46,48,51,61,63,65,67,69,71,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,134,156,165,176,179,189,217,220,222,224,226,228,230,232,234,236,238,240,242,244,246,248,250,252,254,256,258,260,262,264,266,268,270,272,382,397,399,401,419,421,423,432,445,454,494,574,605,613,638,674,690,704,707,710,715,727,738,744,746,803,804,809,810,811,812,820,831,833,835,837,843,904,959,960,961,969,970,971,972,982,1004,1007,1010) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(path)


Rows: 195196 Columns: 1024
